In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb

In [2]:
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix, cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import matthews_corrcoef
from sklearn.model_selection import train_test_split

In [3]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import ExtraTreeClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score, cross_val_predict

C:\Users\User\anaconda3\Lib\site-packages\dask\dataframe\_pyarrow_compat.py:23: UserWarning: You are using pyarrow version 11.0.0 which is known to be insecure. See https://www.cve.org/CVERecord?id=CVE-2023-47248 for further details. Please upgrade to pyarrow>=14.0.1 or install pyarrow-hotfix to patch your current version.
  warnings.warn(


In [4]:
df = pd.read_csv('protine_gtpc.csv')
df.head()

,0,0.052631579,0.052631579.1,0.1,0.2,0.3,0.4,0.5,0.6,0.7,...,0.052631579.13,0.101,0.102,0.052631579.14,0.103,0.104,0.105,0.106,0.107,0.108
0,0,0.000000,0.052632,0.000000,0.052632,0.000000,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.052632,0.0,0.0
1,0,0.052632,0.052632,0.105263,0.000000,0.000000,0.052632,0.0,0.0,0.000000,...,0.0,0.0,0.052632,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0
2,0,0.105263,0.052632,0.105263,0.000000,0.000000,0.000000,0.0,0.0,0.052632,...,0.0,0.0,0.000000,0.0,0.0,0.052632,0.0,0.000000,0.0,0.0
3,0,0.052632,0.000000,0.000000,0.000000,0.052632,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.052632,0.0,0.0
4,0,0.000000,0.000000,0.000000,0.052632,0.105263,0.000000,0.0,0.0,0.000000,...,0.0,0.0,0.000000,0.0,0.0,0.052632,0.0,0.000000,0.0,0.0


In [5]:
y = df['0']
x = df.drop('0', axis=1) 

In [6]:
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.3)

In [7]:
total_Metics = []
total_Metics = pd.DataFrame(total_Metics)
total_Metics['Classifier'] = 'Classifier'
total_Metics['Accuracy'] = 'Accuracy'
total_Metics['mcc'] = 'mcc'
# total_Metics['auc'] = 'auc'
total_Metics['Kappa'] = 'Kappa'
total_Metics['precision'] = 'precision'
total_Metics['recall'] = 'recall'
total_Metics['f1'] = 'f1'
total_Metics['sensitivity'] = 'sensitivity'
total_Metics['specificity'] = 'specificity'

models = [RandomForestClassifier(n_estimators = 200, max_depth = 10),
          XGBClassifier(n_estimators = 200,max_depth = 10, learning_rate = 0.1),
          LGBMClassifier(learning_rate = 0.1,max_depth = 10,random_state = 50),
          GradientBoostingClassifier(n_estimators = 200, learning_rate = 0.5, random_state = 50),
          AdaBoostClassifier(n_estimators = 200, learning_rate = 0.1, random_state = 50)]
cv = KFold(n_splits=5, random_state=1, shuffle=True)
for model in models:
  from sklearn.metrics import f1_score, precision_score, recall_score, log_loss, accuracy_score, matthews_corrcoef, roc_auc_score, cohen_kappa_score
  # evaluate model
  # scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
  # model.fit(xtrain, ytrain)
  # pred = model.predict(xtest)
  pred = cross_val_predict(model, xtrain, ytrain, cv=cv, n_jobs=-1)

  # cm1 = confusion_matrix(y, y_pred)
  # report performance
  Accuracy = accuracy_score(ytrain, pred)
  mcc = matthews_corrcoef(ytrain, pred)
  cm1 = confusion_matrix(ytrain, pred)
  kappa = cohen_kappa_score(ytrain, pred)
  f1 = f1_score(ytrain, pred)
  precision_score = precision_score(ytrain, pred)
  recall_score = recall_score(ytrain, pred)
  sensitivity = cm1[0,0]/(cm1[0,0]+cm1[0,1])
  specificity = cm1[1,1]/(cm1[1,0]+cm1[1,1])
  # y_pred = np.argmax(y_pred, axis=0)
  # auc = roc_auc_score(y, y_pred, multi_class='ovr')
  total_Metics.loc[len(total_Metics.index)] = [model,Accuracy, mcc, kappa, precision_score,recall_score, f1, sensitivity,specificity]

print(total_Metics)

                                          Classifier  Accuracy       mcc  \
0  RandomForestClassifier(max_depth=10, n_estimat...  0.626881 -0.062412   
1  XGBClassifier(base_score=None, booster=None, c...  0.571715 -0.005154   
2      LGBMClassifier(max_depth=10, random_state=50)  0.580742  0.013451   
3  GradientBoostingClassifier(learning_rate=0.5, ...  0.564694  0.018972   
4  AdaBoostClassifier(learning_rate=0.1, n_estima...  0.626881  0.038937   

      Kappa  precision    recall        f1  sensitivity  specificity  
0 -0.025187   0.166667  0.011236  0.021053     0.968799     0.011236  
1 -0.004962   0.352697  0.238764  0.284757     0.756630     0.238764  
2  0.012899   0.368644  0.244382  0.293919     0.767551     0.244382  
3  0.018830   0.370861  0.314607  0.340426     0.703588     0.314607  
4  0.028392   0.414894  0.109551  0.173333     0.914197     0.109551  


In [8]:
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=42)
x_ros, y_ros = ros.fit_resample(xtrain, ytrain)
total_Metics = []
total_Metics = pd.DataFrame(total_Metics)
total_Metics['Classifier'] = 'Classifier'
total_Metics['Accuracy'] = 'Accuracy'
total_Metics['mcc'] = 'mcc'
# total_Metics['auc'] = 'auc'
total_Metics['Kappa'] = 'Kappa'
total_Metics['precision'] = 'precision'
total_Metics['recall'] = 'recall'
total_Metics['f1'] = 'f1'
total_Metics['sensitivity'] = 'sensitivity'
total_Metics['specificity'] = 'specificity'

models = [RandomForestClassifier(n_estimators = 200, max_depth = 10),
          XGBClassifier(n_estimators = 200,max_depth = 10, learning_rate = 0.1),
          LGBMClassifier(learning_rate = 0.1,max_depth = 10,random_state = 50),
          GradientBoostingClassifier(n_estimators = 200, learning_rate = 0.5, random_state = 50),
          AdaBoostClassifier(n_estimators = 200, learning_rate = 0.1, random_state = 50)]
cv = KFold(n_splits=5, random_state=1, shuffle=True)
for model in models:
  from sklearn.metrics import f1_score, precision_score, recall_score, log_loss, accuracy_score, matthews_corrcoef, roc_auc_score, cohen_kappa_score
  # evaluate model
  # scores = cross_val_score(model, X, y, scoring='accuracy', cv=cv, n_jobs=-1)
  # model.fit(xtrain, ytrain)
  # pred = model.predict(xtest)
  pred = cross_val_predict(model, x_ros, y_ros, cv=cv, n_jobs=-1)

  # cm1 = confusion_matrix(y, y_pred)
  # report performance
  Accuracy = accuracy_score(y_ros, pred)
  mcc = matthews_corrcoef(y_ros, pred)
  cm1 = confusion_matrix(y_ros, pred)
  kappa = cohen_kappa_score(y_ros, pred)
  f1 = f1_score(y_ros, pred)
  precision_score = precision_score(y_ros, pred)
  recall_score = recall_score(y_ros, pred)
  sensitivity = cm1[0,0]/(cm1[0,0]+cm1[0,1])
  specificity = cm1[1,1]/(cm1[1,0]+cm1[1,1])
  # y_pred = np.argmax(y_pred, axis=0)
  # auc = roc_auc_score(y, y_pred, multi_class='ovr')
  total_Metics.loc[len(total_Metics.index)] = [model,Accuracy, mcc, kappa, precision_score,recall_score, f1, sensitivity,specificity]

print(total_Metics)

                                          Classifier  Accuracy       mcc  \
0  RandomForestClassifier(max_depth=10, n_estimat...  0.712168  0.425208   
1  XGBClassifier(base_score=None, booster=None, c...  0.726989  0.455848   
2      LGBMClassifier(max_depth=10, random_state=50)  0.728549  0.460009   
3  GradientBoostingClassifier(learning_rate=0.5, ...  0.703588  0.412431   
4  AdaBoostClassifier(learning_rate=0.1, n_estima...  0.587363  0.174988   

      Kappa  precision    recall        f1  sensitivity  specificity  
0  0.424337   0.699413  0.744150  0.721088     0.680187     0.744150  
1  0.453978   0.708155  0.772231  0.738806     0.681747     0.772231  
2  0.457098   0.705470  0.784711  0.742984     0.672387     0.784711  
3  0.407176   0.675639  0.783151  0.725434     0.624025     0.783151  
4  0.174727   0.582840  0.614665  0.598330     0.560062     0.614665  
